# Задание 1

## Подготовка данных

In [69]:
!pip install russian-tagsets

  Created wheel for russian-tagsets: filename=russian_tagsets-0.6-py3-none-any.whl size=24640 sha256=851f3ee5a0bf799a0f0716083a6bf434c8c2637e3f90aa9fcd6db74640ab50d1
  Stored in directory: c:\users\ruthenian8\appdata\local\pip\cache\wheels\12\2e\54\71c28ef06e79d9bdd7843ad80473900615056abb3261544039
Successfully built russian-tagsets


In [71]:
from russian_tagsets import converters

In [1]:
import pandas as pd
from lxml import html
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
from razdel import tokenize as razdel_tokenize
import os
import lxml
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
import nltk
nltk.download("stopwords")
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    
    tokens = [token.text for token in list(razdel_tokenize(text))]
    tokens = [token for token in tokens if token.isalnum()]

    return ' '.join(tokens)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ruthenian8\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
!wget "http://vectors.nlpl.eu/repository/20/180.zip"
!unzip "180.zip"

--2021-02-07 18:32:44--  http://vectors.nlpl.eu/repository/20/180.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 484452317 (462M) [application/zip]
Saving to: ‘180.zip’

180.zip             100%[===================>] 462.01M  23.3MB/s    in 21s     

2021-02-07 18:33:06 (22.3 MB/s) - ‘180.zip’ saved [484452317/484452317]

Archive:  180.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [2]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [72]:
to_ud = converters.converter('opencorpora-int', "ud14")

In [118]:
def gud_normalize(text):
    words = text.split()
    tokens = []
    for word in words:
        analysis = morph.parse(word)[0]
        lemma = analysis.normal_form
        tag = analysis.tag
            
        if tag._is_unknown() == False:
            pos = to_ud(str(tag.POS))
            pos = pos[:pos.find(" ")]
        else:
            pos = 'UNKN'            
        tokens.append(lemma+'_'+pos)
    return tokens


In [8]:
!unzip "paraphraser.zip"

Archive:  paraphraser.zip
  inflating: LICENSE                 
  inflating: corpus.xml              
  inflating: paraphrases.xml         


In [8]:
with open("paraphrases.xml", "r", encoding="UTF-8") as file:
    corp = file.read()

In [21]:
soup = BeautifulSoup(corp, "xml")
pars = soup.find_all("paraphrase")
dicts = []
for index in range(len(pars)):
    parent = pars[index]
    mapp = {}
    mapp["text_1"] = parent.find(name="value", attrs={"name": "text_1"}).text.lower()
    mapp["text_2"] = parent.find(name="value", attrs={"name": "text_2"}).text.lower()
    mapp["class"] = parent.find(name="value", attrs={"name": "class"}).text
    dicts.append(mapp)

In [22]:
paraphrases = pd.DataFrame.from_dict(dicts)
paraphrases.head(5)

,text_1,text_2,class
0,полицейским разрешат стрелять на поражение по ...,полиции могут разрешить стрелять по хулиганам ...,0
1,право полицейских на проникновение в жилище ре...,правила внесудебного проникновения полицейских...,0
2,президент египта ввел чрезвычайное положение в...,власти египта угрожают ввести в стране чрезвыч...,0
3,вернувшихся из сирии россиян волнует вопрос тр...,самолеты мчс вывезут россиян из разрушенной си...,-1
4,в москву из сирии вернулись 2 самолета мчс с р...,самолеты мчс вывезут россиян из разрушенной си...,0


In [11]:
data = open('wiki_data.txt', encoding="utf-8").read().splitlines()

data_norm = [normalize(tokenize(text)) for text in data]

In [12]:
data_norm = [text for text in data_norm if text]

## Обучение векторов

In [13]:
word2vec = gensim.models.Word2Vec([text.split() for text in data_norm], size=300, sg=1)

In [32]:
veri_gud_markers = {"NOUN", "VERB", "ADJ", "ADV"}

In [124]:
def get_mean_vector_custom(string, model):
    words = string.split()
    init = np.zeros(300)
    for word in words:
        try:
            vec = model[word]
            init += vec
        except:
            continue
    return init / len(words)
            
def get_mean_vector_pretrained(string, model):
    words = gud_normalize(string)
    init = np.zeros(300)
    for word in words:
        try:
            vec = model[word]
            gram_marker = word[word.find("_")+1:]
            if gram_marker in veri_gud_markers:
                init += vec * 0.8
            else:
                init += vec * 0.2
        except:
            continue
    return init / len(words)

In [48]:
paraphrases["custom"] = np.zeros(paraphrases.shape[0])
paraphrases["pretrained"] = np.zeros(paraphrases.shape[0])

In [59]:
custom_first = []
custom_second = []
pretrained_first = []
pretrained_second = []

In [125]:
for index in tqdm(range(paraphrases.shape[0])):
    t1 = paraphrases.loc[index, "text_1"]
    t2 = paraphrases.loc[index, "text_2"]
    custom_first.append(get_mean_vector_custom(t1, word2vec))
    custom_second.append(get_mean_vector_custom(t2, word2vec))
                                                       
    pretrained_first.append(get_mean_vector_pretrained(t1, model))
    pretrained_second.append(get_mean_vector_pretrained(t2, model))

  0%|                                                                                         | 0/7227 [00:00<?, ?it/s]<ipython-input-124-1b59c036ecfc>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vec = model[word]
100%|█████████████████████████████████████████████████████████████████████████████| 7227/7227 [00:09<00:00, 728.60it/s]


In [128]:
paraphrases["custom"] = [np.concatenate([a, b]) for a, b in zip(custom_first, custom_second)]
paraphrases["pretrained"] = [np.concatenate([a, b]) for a, b in zip(pretrained_first, pretrained_second)]

## Оценка показателей

In [129]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [144]:
X_custom = np.array(paraphrases["custom"].to_list())
X_pretrained = np.array(paraphrases["pretrained"].to_list())
y = [int(i) for i in paraphrases["class"]]

### Показатели для кастомных векторов

In [151]:
log_1 = LogisticRegression(class_weight='balanced', C=1, n_jobs=3)
log_2 = LogisticRegression(class_weight='balanced', C=10, n_jobs=3)
cross_val_score(log_1, X_custom, y, scoring="f1_macro", cv=StratifiedKFold(n_splits=5, shuffle=True))

array([0.42350141, 0.42715905, 0.41458905, 0.42439358, 0.41451315])

### Для готовых векторов

In [152]:
cross_val_score(log_2, X_pretrained, y, scoring="f1_macro", cv=StratifiedKFold(n_splits=5, shuffle=True))

array([0.41686953, 0.42845478, 0.43880293, 0.42791729, 0.39506934])

# Задание 2

## Обучение векторов

In [153]:
fasttext = gensim.models.FastText([text.split() for text in data_norm], size=300)

In [156]:
ft_first = [get_mean_vector_custom(i, fasttext) for i in paraphrases["text_1"]]
ft_second = [get_mean_vector_custom(i, fasttext) for i in paraphrases["text_2"]]

<ipython-input-124-1b59c036ecfc>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vec = model[word]


In [209]:
cv = CountVectorizer(min_df=4, max_df=0.4, max_features=6000)
vocab = np.concatenate((paraphrases["text_1"], paraphrases["text_2"])).tolist()
for index in range(len(vocab)):
    vocab[index] = normalize(vocab[index])
X_matrix = cv.fit_transform(vocab)

In [210]:
id2word = {i:w for i,w in enumerate(cv.get_feature_names())}
word2id = {w:i for i,w in id2word.items()}

In [211]:
svd = TruncatedSVD(300)
svd.fit(X_matrix)

TruncatedSVD(n_components=300)

In [212]:
nmf = NMF(300)
nmf.fit(X_matrix)

NMF(n_components=300)

In [213]:
id2vec_nmf = nmf.components_.T
id2vec_svd = svd.components_.T

In [214]:
def mean_for_dim_reduction(string, id2vec):
    words = string.split()
    init = np.zeros(300)
    len_count = 0
    for word in words:
        lemma = morph.parse(word)[0].normal_form
        try:
            vec = id2vec[word2id[lemma]]
            init += vec
            len_count += 1
        except:
            continue
    if len_count == 0:
        return init
    return init / len_count

In [215]:
svd_first = [mean_for_dim_reduction(i, id2vec_svd) for i in paraphrases["text_1"]]
svd_second = [mean_for_dim_reduction(i, id2vec_svd) for i in paraphrases["text_2"]]
nmf_first = [mean_for_dim_reduction(i, id2vec_nmf) for i in paraphrases["text_1"]]
nmf_second = [mean_for_dim_reduction(i, id2vec_nmf) for i in paraphrases["text_2"]]

In [175]:
from sklearn.metrics.pairwise import cosine_similarity

In [177]:
cs_custom = [cosine_similarity(a.reshape(1, -1), b.reshape(1, -1)) for a, b in zip(custom_first, custom_second)]
cs_pr = [cosine_similarity(a.reshape(1, -1), b.reshape(1, -1)) for a, b in zip(pretrained_first, pretrained_second)]
cs_ft = [cosine_similarity(a.reshape(1, -1), b.reshape(1, -1)) for a, b in zip(ft_first, ft_second)]

In [216]:
cs_svd = [cosine_similarity(a.reshape(1, -1), b.reshape(1, -1)) for a, b in zip(svd_first, svd_second)]
cs_nmf = [cosine_similarity(a.reshape(1, -1), b.reshape(1, -1)) for a, b in zip(nmf_first, nmf_second)]

In [217]:
paraphrases["cs_custom"] = cs_custom
paraphrases["cs_pr"] = cs_pr
paraphrases["cs_ft"] = cs_ft
paraphrases["cs_svd"] = cs_svd
paraphrases["cs_nmf"] = cs_nmf

## Оценка показателей

In [219]:
X_num = paraphrases[["cs_custom", "cs_pr", "cs_ft", "cs_svd", "cs_nmf"]]
y_num = paraphrases["class"]

In [231]:
log_3 = LogisticRegression(l1_ratio=0.2, class_weight='balanced', solver='saga', penalty="elasticnet", C=10, max_iter=130, n_jobs=3)
cross_val_score(log_3, X_num, y_num, scoring="f1_micro", cv=StratifiedKFold(n_splits=5, shuffle=True))

array([0.47164592, 0.48893499, 0.50519031, 0.49273356, 0.47958478])

## Корректировки в методах векторизации

In [232]:
word2vec = gensim.models.Word2Vec([text.split() for text in data_norm], window=6, min_count=3, size=300, sg=1)

In [235]:
custom_first = []
custom_second = []
for index in tqdm(range(paraphrases.shape[0])):
    t1 = paraphrases.loc[index, "text_1"]
    t2 = paraphrases.loc[index, "text_2"]
    custom_first.append(get_mean_vector_custom(t1, word2vec))
    custom_second.append(get_mean_vector_custom(t2, word2vec))

  0%|                                                                                         | 0/7227 [00:00<?, ?it/s]<ipython-input-124-1b59c036ecfc>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vec = model[word]
100%|████████████████████████████████████████████████████████████████████████████| 7227/7227 [00:01<00:00, 5677.17it/s]


In [236]:
cs_custom = [cosine_similarity(a.reshape(1, -1), b.reshape(1, -1)) for a, b in zip(custom_first, custom_second)]
paraphrases["cs_custom"] = cs_custom
X_num = paraphrases[["cs_custom", "cs_pr", "cs_ft", "cs_svd", "cs_nmf"]]
y_num = paraphrases["class"]

In [240]:
log_3 = LogisticRegression(l1_ratio=0.2,
                           class_weight='balanced',
                           solver='saga',
                           penalty="elasticnet",
                           C=10, max_iter=120,
                           n_jobs=3)
cross_val_score(log_3,
                X_num,
                y_num,
                scoring="f1_micro",
                cv=StratifiedKFold(n_splits=5,
                                   shuffle=True))

array([0.47164592, 0.5055325 , 0.48442907, 0.48581315, 0.49134948])

In [247]:
# Изменил корректировку весов в зависимости от части речи в обеих функциях
# Увеличены веса для важных и неважных слов
# Слова вне словаря не учитываются при подсчете среднего
#
def get_mean_vector_custom(string, model):
    words = gud_normalize(string)
    init = np.zeros(300)
    len_count = 0
    for word in words:
        try:
            chunk = word[:word.find("_")]
            vec = model[chunk]
            gram_marker = word[word.find("_")+1:]
            if gram_marker in veri_gud_markers:
                init += vec * 1.2
            else:
                init += vec * 0.6
            len_count += 1
        except:
            continue
    if len_count == 0:
        return init
    return init / len_count
            
def get_mean_vector_pretrained(string, model):
    words = gud_normalize(string)
    init = np.zeros(300)
    len_count = 0
    for word in words:
        try:
            vec = model[word]
            gram_marker = word[word.find("_")+1:]
            if gram_marker in veri_gud_markers:
                init += vec * 1.2
            else:
                init += vec * 0.6
            len_count += 1
        except:
            continue
    if len_count == 0:
        return init
    return init / len_count

In [251]:
custom_first = []
custom_second = []
pretrained_first = []
pretrained_second = []

In [252]:
for index in tqdm(range(paraphrases.shape[0])):
    t1 = paraphrases.loc[index, "text_1"]
    t2 = paraphrases.loc[index, "text_2"]
    custom_first.append(get_mean_vector_custom(t1, word2vec))
    custom_second.append(get_mean_vector_custom(t2, word2vec))
                                                       
    pretrained_first.append(get_mean_vector_pretrained(t1, model))
    pretrained_second.append(get_mean_vector_pretrained(t2, model))

  0%|                                                                                         | 0/7227 [00:00<?, ?it/s]<ipython-input-247-4a8730930e54>:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vec = model[chunk]
100%|█████████████████████████████████████████████████████████████████████████████| 7227/7227 [00:17<00:00, 416.81it/s]


In [257]:
cs_custom = [cosine_similarity(a.reshape(1, -1), b.reshape(1, -1)) for a, b in zip(custom_first, custom_second)]
cs_pr = [cosine_similarity(a.reshape(1, -1), b.reshape(1, -1)) for a, b in zip(pretrained_first, pretrained_second)]
paraphrases["cs_custom"] = cs_custom
paraphrases["cs_pr"] = cs_pr
X_num = paraphrases[["cs_custom", "cs_pr", "cs_ft", "cs_svd", "cs_nmf"]]
y_num = paraphrases["class"]

## Оценка показателей

In [262]:
log_3 = LogisticRegression(l1_ratio=0.2,
                           class_weight='balanced',
                           solver='saga',
                           penalty="elasticnet",
                           C=10, max_iter=120,
                           n_jobs=3)
cross_val_score(log_3,
                X_num,
                y_num,
                scoring="f1_micro",
                cv=StratifiedKFold(n_splits=5,
                                   shuffle=True))

array([0.53526971, 0.52558783, 0.52179931, 0.52941176, 0.54256055])

### Комментарий: по сравнению с первоначальной моделью отмечается небольшое улучшение качества (~0.05).